In [1]:
%pip install --extra-index-url=https://packages.dea.ga.gov.au/ fc


Looking in indexes: https://pypi.org/simple, https://packages.dea.ga.gov.au/
  Using cached fc-1.3.9-cp310-cp310-linux_x86_64.whl
  Using cached https://packages.dea.ga.gov.au/digitalearthau/digitalearthau-20220517-py3-none-any.whl (278 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import warnings
import datacube
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt

from fc.fractional_cover import (
    fractional_cover,
)  # import the FC package after installing it above
from odc.geo.geom import Geometry

import sys

sys.path.insert(1, "/home/jovyan/dev/Tools/")
from dea_tools.datahandling import load_ard
from dea_tools.dask import create_local_dask_cluster
from dea_tools.plotting import rgb

warnings.filterwarnings("ignore")


In [ ]:
client = create_local_dask_cluster(return_client=True)


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/jenna.guffogg@ga.gov.au/proxy/8787/status,
Dashboard: /user/jenna.guffogg@ga.gov.au/proxy/8787/status,Workers: 1
Total threads: 15,Total memory: 114.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38243,Workers: 1
Dashboard: /user/jenna.guffogg@ga.gov.au/proxy/8787/status,Total threads: 15
Started: Just now,Total memory: 114.00 GiB
Comm: tcp://127.0.0.1:43591,Total threads: 15
Dashboard: /user/jenna.guffogg@ga.gov.au/proxy/43951/status,Memory: 114.00 GiB
Nanny: tcp://127.0.0.1:40921,


2025-08-06 03:58:33,595 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:43591' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('stack-3522098c869efc85a4f395c53050e127', 0, 0, 1, 1), ('dc_load_nbart_green-2ac7dcc351a0443f963b94657eac7728', 1, 1, 0), ('stack-3522098c869efc85a4f395c53050e127', 2, 0, 1, 0), 'dataset-f0a033f174dd44acba7ad12ef063daed', ('dc_load_nbart_blue-a9f7e7174169481e92f9b44876b6c240', 1, 0, 1), ('dc_load_nbart_green-2ac7dcc351a0443f963b94657eac7728', 1, 0, 1), ('stack-3522098c869efc85a4f395c53050e127', 2, 0, 0, 1), ('stack-3522098c869efc85a4f395c53050e127', 1, 0, 0, 0), ('dc_load_nbart_red-b84e1293a84c4b43bdb95b0eaa77dc0e', 1, 1, 0), ('stack-3522098c869efc85a4f395c53050e127', 0, 0, 1, 0), ('stack-3522098c869efc85a4f395c53050e127', 1, 0, 1, 1), 'dataset-108e7c2acd4340d1b1a7da1dd47d88de', ('dc_load_nbart_red-b84e1293a84c4b43bdb95b0eaa77dc0e', 1, 0, 1), ('stack-3522098c869efc85a4f395c53050e127', 0, 0, 0, 1), 

In [ ]:
# region_code = ['x176y085'] #marysville
# region_code = ['x168y092'] #hopetoun
# region_code = ['x140y138'] #west macdonnell
# region_codes = ['x148y166'] #limmen NT - cloud heavy area
# region_codes = ['x175y066'] # Tas SW nat park


region_codes = ["x176y085"]

time = ("2022-01-01", "2022-03-01")

resolution = (-30, 30)
filters_id = "test"
measurements = [
    "nbart_green",
    "nbart_red",
    "nbart_blue",
    "nbart_nir",
    "nbart_swir_1",
    "nbart_swir_2",
]
dask_chunks = dict(x=1024, y=1024, time=1)


In [ ]:
# open tiles and select

gdf = gpd.read_file(
    "~/gdata1/projects/fc-sub-annual/data/testing_minitile_suite.geojson"
)

gdf = gdf[gdf["region_code"].isin(region_codes)]

geom = Geometry(geom=gdf.iloc[0].geometry, crs=gdf.crs)


In [ ]:
dc = datacube.Datacube(app="fc_ls_test")

query = {
    "time": time,
    "measurements": measurements,
    "resolution": resolution,
    "geopolygon": geom,
    "dask_chunks": dask_chunks,
    "group_by": "solar_day",
    "output_crs": "EPSG:3577",
}


In [ ]:
# gdf.explore(
#     tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#     attr='Esri',
#     name='Esri satellite'
# )


In [ ]:
ls_fc = load_ard(
    dc=dc,
    products=["ga_ls8c_ard_3"],
    cloud_mask="fmask",
    mask_pixel_quality=True,
    mask_contiguity=True,
    skip_broken_datasets=True,
    verbose=True,
    **query,
)


Finding datasets
    ga_ls8c_ard_3
Applying fmask pixel quality/cloud mask
Applying contiguity mask (oa_nbart_contiguity)
Returning 4 time steps as a dask array


In [ ]:
rgb(
    ls_fc,
    bands=["nbart_blue", "nbart_green", "nbart_red"],
    col_wrap=4,
    robust=True,
    size=3,
)


KeyboardInterrupt: 

In [ ]:
# def fc_run_function(dc, geom, query, time, region_code):
#     # update the query to include our geometry
#     query.update({"geopolygon": geom})

#     # load available data from landsat
#     ls_fc = load_ard(
#         dc=dc,
#         products=["ga_ls9c_ard_3", "ga_ls8c_ard_3"],
#         cloud_mask="fmask",
#         mask_pixel_quality=True,
#         mask_contiguity=True,
#         skip_broken_datasets=True,
#         verbose=False,
#         **query,
#     )
#     # -----fractional-cover-----#

#     # -----export-tiles-----#
#     tiles_folder = f"/gdata/projects/fc-sub-annual/results/{filters_id}"
#     if not os.path.exists(tiles_folder):
#         os.makedirs(tiles_folder)
